In [1]:
from torch.ao.nn.quantized import FloatFunctional

'''
Properly implemented ResNet-s for CIFAR10 as described in paper [1].

The implementation and structure of this file is hugely influenced by [2]
which is implemented for ImageNet and doesn't have option A for identity.
Moreover, most of the implementations on the web is copy-paste from
torchvision's resnet and has wrong number of params.

Proper ResNet-s for CIFAR10 (for fair comparision and etc.) has following
number of layers and parameters:

name      | layers | params
ResNet20  |    20  | 0.27M
ResNet32  |    32  | 0.46M
ResNet44  |    44  | 0.66M
ResNet56  |    56  | 0.85M
ResNet110 |   110  |  1.7M
ResNet1202|  1202  | 19.4m

which this implementation indeed has.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
[2] https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

If you use this implementation in you work, please don't forget to mention the
author, Yerlan Idelbayev.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.quantization import QuantStub, DeQuantStub

from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='B'):
        super(BasicBlock, self).__init__()
        self.ff = FloatFunctional() # To make it ready for quantization

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A': # Option A does not work with torch-pruning, as the LambdaLayer cannot be analysed. Therefore we use option B
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.ff.add(out, self.shortcut(x)) # remove + for quantization
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.quant(x) # add quant
        out = F.relu(self.bn1(self.conv1(out)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = self.dequant(out) # add dequant
        return out

    def fuse_model(self):
        torch.quantization.fuse_modules(self, [['conv1', 'bn1']], inplace=True)
        for m in self.layer1:
            torch.quantization.fuse_modules(m, [['conv1', 'bn1'], ['conv2', 'bn2']], inplace=True)
        for m in self.layer2:
            torch.quantization.fuse_modules(m, [['conv1', 'bn1'], ['conv2', 'bn2']], inplace=True)
        for m in self.layer3:
            torch.quantization.fuse_modules(m, [['conv1', 'bn1'], ['conv2', 'bn2']], inplace=True)



def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])


def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])


def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])


def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])


def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])


def resnet1202():
    return ResNet(BasicBlock, [200, 200, 200])


def test(net):
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


#if __name__ == "__main__":
#    for net_name in __all__:
#        if net_name.startswith('resnet'):
#            print(net_name)
#            test(globals()[net_name]())
#            print()

In [2]:
import torch
import torch_pruning as tp

model = resnet110()

# Importance criteria
example_inputs = torch.randn(1, 3, 32, 32)
imp = tp.importance.TaylorImportance()

ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 10:
        ignored_layers.append(m) # DO NOT prune the final classifier!

iterative_steps = 5 # progressive pruning
pruner = tp.pruner.MagnitudePruner(
    model,
    example_inputs,
    importance=imp,
    iterative_steps=iterative_steps,
    ch_sparsity=0.34,
    ignored_layers=ignored_layers,
)

base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
for i in range(iterative_steps):
    if isinstance(imp, tp.importance.TaylorImportance):
        # Taylor expansion requires gradients for importance estimation
        loss = model(example_inputs).sum() # a dummy loss for TaylorImportance
        loss.backward() # before pruner.step()
    pruner.step()
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    # finetune your model here
    # finetune(model)
    # ...
    # Calculate the difference in parameters between the original and pruned model
    #print(f"After pruning step {i + 1}:")
    #print(f"Number of parameters: {nparams}")
    #print(f"Difference in parameters: {base_nparams - nparams}\n")

/Users/christophknaden/git/model-compression/.venv/lib/python3.11/site-packages/torch_pruning/pruner/algorithms/base_pruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn(


In [3]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.utils.data

def get_cifar10_loader(split='train', batch_size=128, workers=0, subset_size=0):
    """
    Args:
        split (str): 'train' or 'val'
        batch_size (int): Batch size for DataLoader
        workers (int): Number of worker threads
        subset_size (int): If > 0, return a subset of the dataset
    """
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    if split == 'train':
        dataset = datasets.CIFAR10(
            root='./data', train=True,
            transform=transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomCrop(32, 4),
                transforms.ToTensor(),
                normalize,
            ]), download=True)
        shuffle = True
    elif split == 'val':
        dataset = datasets.CIFAR10(
            root='./data', train=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                normalize,
            ]), download=True)
        shuffle = False
    else:
        raise ValueError("split must be either 'train' or 'val'")

    if subset_size > 0:
        dataset = torch.utils.data.Subset(dataset, range(subset_size))

    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size,
        shuffle=shuffle, num_workers=workers,
        pin_memory=True)

    return loader


In [4]:
def evaluate(model, val_loader, device, use_half=False):
    import time
    import torch.nn as nn

    criterion = nn.CrossEntropyLoss().to(device)
    model.eval().to(device)

    correct = 0
    total = 0
    total_loss = 0
    start = time.time()

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            if use_half:
                inputs = inputs.half()
                criterion = criterion.half()

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            total_loss += loss.item() * targets.size(0)

    acc = 100. * correct / total
    avg_loss = total_loss / total
    elapsed = time.time() - start

    print(f"Validation Accuracy: {acc:.2f}%, Avg Loss: {avg_loss:.4f}, Time: {elapsed:.2f}s")
    return acc, avg_loss


In [5]:
from tqdm import tqdm

def train_model(model, train_loader, optimizer, criterion, device, num_epochs=10, use_half=False):
    model.to(device)
    if use_half:
        model = model.half()
        criterion = criterion.half()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)

            if use_half:
                inputs = inputs.half()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            acc = 100. * correct / total
            avg_loss = running_loss / (progress_bar.n + 1)
            progress_bar.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{acc:.2f}%")


In [6]:
import time
import torch

def measure_inference_time(model, dataloader, num_batches=100):
    model.eval()
    total_time = 0.0

    with torch.inference_mode():
        for i, (inputs, _) in enumerate(dataloader):
            if i >= num_batches:
                break

            start_time = time.time()
            _ = model(inputs)
            end_time = time.time()

            total_time += (end_time - start_time)

    avg_time_per_batch = total_time / num_batches
    return avg_time_per_batch


In [7]:
#model = resnet110()
model.to("mps")
device = torch.device("mps")


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()
train_loader = get_cifar10_loader('train', batch_size=128)
val_loader = get_cifar10_loader('val', batch_size=128)
val_loader_subset = get_cifar10_loader('val', batch_size=128, subset_size=1000)


In [7]:
train_model(model, train_loader, optimizer, criterion, device, num_epochs=10, use_half=False)


In [8]:
evaluate(model, val_loader, device)

Validation Accuracy: 76.93%, Avg Loss: 0.7282, Time: 5.58s


(76.93, 0.7282403444290161)

In [9]:
# Save the full pruned model after all pruning steps
torch.save(model, "resnet110.pth")

In [9]:
# Load the full pruned model
pruned_model = torch.load("pruned_34-30_resnet110.pth", map_location=torch.device('cpu'), weights_only=False)

# Set the model to evaluation mode (or training mode as needed)
pruned_model.eval()
print("Pruned model loaded successfully.")


Pruned model loaded successfully.


In [7]:
evaluate(pruned_model, val_loader, device)

NameError: name 'val_loader' is not defined

In [10]:
# Count the total number of parameters
total_params = sum(p.numel() for p in pruned_model.parameters())
print(f"Total number of parameters in the pruned model: {total_params}")


Total number of parameters in the pruned model: 743997


In [15]:
pruned_model.to("cpu")
device = torch.device("cpu")

torch.backends.quantized.engine = 'qnnpack'

model_fp32 = pruned_model
model_fp32.eval()

model_fp32.fuse_model()

# Sets the backend for x86
model_fp32.qconfig = torch.quantization.get_default_qconfig('qnnpack')

# Prepares the model for the next step i.e. calibration.
# Inserts observers in the model that will observe the activation tensors during calibration
model_fp32_prepared = torch.quantization.prepare(model_fp32, inplace = False)

evaluate(model_fp32_prepared, val_loader_subset, device)

model_quantized = torch.quantization.convert(model_fp32_prepared)

Validation Accuracy: 85.70%, Avg Loss: 0.4356, Time: 3.05s


In [16]:
evaluate(model_quantized, val_loader, device)

Validation Accuracy: 85.36%, Avg Loss: 0.4500, Time: 19.02s


(85.36, 0.45002267847061156)

In [18]:
pruned_model.to("cpu")
model_quantized.to("cpu")


time_float = measure_inference_time(pruned_model, val_loader)
time_quant = measure_inference_time(model_quantized, val_loader)

print(f"Average inference time per batch (float model): {time_float:.4f} seconds")
print(f"Average inference time per batch (quantized model): {time_quant:.4f} seconds")


Average inference time per batch (float model): 0.2189 seconds
Average inference time per batch (quantized model): 0.2050 seconds
